## Tailpatch for Gutenberg experiments

In [ ]:
## TailPatch Function: With Log Likelihood
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torch.nn as nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.optim import AdamW
import numpy as np
import random
import gc
import math
from torch.utils.data import TensorDataset



import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
device = torch.device("cpu")

import random
import numpy as np
import torch

def tp_debug_log(samples,output_ids, prompt_length=1):
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)  

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    dataloader = DataLoader(
        samples,
        batch_size=1,
        shuffle=False
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    device = 'cpu'
    model = GPT2LMHeadModel.from_pretrained('out/gpt2-scratch-mixed')
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=1e-5)
    with torch.no_grad():
        model_pass = model(output_ids, labels=output_ids)

    logits = model_pass.logits
    logits = logits[:, prompt_length-1:-1, :]
    log_probs = torch.log_softmax(logits, dim=-1)
    target_ids = output_ids[:, prompt_length:]
    token_probs = log_probs.gather(2, target_ids.unsqueeze(-1))
    token_probs = token_probs.squeeze(-1)
    print(tokenizer.decode(target_ids[0], skip_special_tokens=True))
    original_probability = 0
    for i in range(token_probs.shape[1]):
        original_probability += token_probs[0,i]
    original_probability = math.exp(original_probability.item())
    print('Original Probability:', original_probability)
    original_probs = token_probs.clone()



    model.train()
    optimizer = AdamW(model.parameters(), lr=1e-5)
    optimizer.zero_grad()
    for i, batch in enumerate(dataloader):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = input_ids.clone()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()

        del input_ids, attention_mask, outputs, labels, loss
        gc.collect()
        torch.cuda.empty_cache()

    optimizer.step()
    optimizer.zero_grad()

    model.eval()
    
    with torch.no_grad():
        model_pass = model(output_ids, labels=output_ids)

    logits = model_pass.logits
    logits = logits[:, prompt_length-1:-1, :]
    log_probs = torch.log_softmax(logits, dim=-1)
    target_ids = output_ids[:, prompt_length:]
    token_probs = log_probs.gather(2, target_ids.unsqueeze(-1))
    token_probs = token_probs.squeeze(-1)
    probability = 1
    for i in range(token_probs.shape[1]):
        probability += token_probs[0,i]
    probability = math.exp(probability.item())   
    return probability, original_probability, token_probs, original_probs

In [ ]:
## Base Model: gpt2-scratch-mixed
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model = GPT2LMHeadModel.from_pretrained("out/gpt2-scratch-mixed")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model.eval()
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(model.num_parameters())

In [ ]:
import json
with open('selected_dataset_mixed.json', 'r') as f:
    data = json.load(f)

training_data = data['train_data_np']
training_data_authors = data['train_data_authors']
training_data_titles = data['train_data_titles']
print(data.keys())

In [ ]:

import os
import numpy as np

# List and sort checkpoint files
## As a proxy, to get the target sequence which is stored along with the gradient descent optimized model. could also use the unlearned model. 
ckpt_dir = 'out/gutenberg_experiments/finetuned_models'
ckpt_files = sorted(os.listdir(ckpt_dir))
print("Checkpoint files:", ckpt_files)

# List and sort finetuned loss files
finetuned_dir = 'data/losses/gutenberg/experiments_finetuned'
finetuned_losses = sorted(os.listdir(finetuned_dir))
print("Finetuned loss files:", finetuned_losses)

# List and sort unlearned loss files
unlearned_dir = 'data/losses/gutenberg/experiments_unlearned'
unlearned_losses = sorted(os.listdir(unlearned_dir))
print("Unlearned loss files:", unlearned_losses)
assert len(finetuned_losses) == len(unlearned_losses)
assert len(ckpt_files) == len(finetuned_losses)


In [ ]:

from rank_bm25 import BM25Okapi
import torch
import json
with open('selected_dataset_mixed_decoded.json', 'r') as f:
    corpus = json.load(f)

print(f"Loaded {len(corpus)} documents.")
print("Sample:", corpus[0][:300])
corpus = np.array(corpus)

print(len(corpus))
tokenized_corpus = [doc.lower().split() for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)



In [ ]:
import pandas as pd
dfs = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for k, file_path in enumerate(ckpt_files):
    ckpt = torch.load(os.path.join(ckpt_dir, file_path), map_location=device)
    output_ids = ckpt['output_ids']
    prompt_length = ckpt['prompt_length']
    losses_ft = np.load(os.path.join(finetuned_dir, finetuned_losses[k]))
    losses_unl = np.load(os.path.join(unlearned_dir, unlearned_losses[k]))
    losses_diff = np.abs(losses_ft - losses_unl)
    sorted_indices = np.argsort(losses_diff)[::-1]
    print(tokenizer.decode(ckpt['output_ids'][0]))
    samples = []
    for i in range(20):
        samples.append({
            'input_ids': torch.tensor([training_data[int(sorted_indices[i])]]),
            'attention_mask': torch.tensor([1] * len(training_data[int(sorted_indices[i])])),
            'labels': torch.tensor([training_data[int(sorted_indices[i])]])
        })
    p_1, original, _, _ = tp_debug_log(samples[:1], output_ids, prompt_length=ckpt['prompt_length'])
    p_3, _, _, _ = tp_debug_log(samples[:3], output_ids, prompt_length=ckpt['prompt_length'])
    p_5, _, _, _ = tp_debug_log(samples[:5], output_ids, prompt_length=ckpt['prompt_length'])
    p_7, _, _, _ = tp_debug_log(samples[:7], output_ids, prompt_length=ckpt['prompt_length'])
    p_10, _, _, _ = tp_debug_log(samples[:10], output_ids, prompt_length=ckpt['prompt_length'])
    p_15, _, _, _ = tp_debug_log(samples[:15], output_ids, prompt_length=ckpt['prompt_length'])
    p_20, _, _, _ = tp_debug_log(samples[:20], output_ids, prompt_length=ckpt['prompt_length'])
    dfs.append({
        'name': file_path,
        'method': 'FT & UN',
        'p_1': p_1,
        'p_3': p_3,
        'p_5': p_5,
        'p_7': p_7,
        'p_10': p_10,
        'p_15': p_15,
        'p_20': p_20,
        
    })
    dfs.append({
        'name': file_path,
        'method': 'Original',
        'p_1': original,
        'p_3': original,
        'p_5': original,
        'p_7': original,
        'p_10': original,
        'p_15': original,
        'p_20': original,
    })
    random_samples_ = np.random.choice(len(training_data), size=20)
    random_samples = []
    for i in range(20):
        random_samples.append({
            'input_ids': torch.tensor(training_data[random_samples_[i]]),
            'attention_mask': torch.ones_like(torch.tensor(training_data[random_samples_[i]])),
            'labels': torch.tensor(training_data[random_samples_[i]])
        })
    p1, original,_ ,_ = tp_debug_log(random_samples[:1], output_ids, prompt_length)
    p3, original,_ ,_ = tp_debug_log(random_samples[:3], output_ids, prompt_length)
    p5, original,_ ,_ = tp_debug_log(random_samples[:5], output_ids, prompt_length)
    p7, original,_ ,_ = tp_debug_log(random_samples[:7], output_ids, prompt_length)
    p10, original,_ ,_ = tp_debug_log(random_samples[:10], output_ids, prompt_length)
    p15, original,_ ,_ = tp_debug_log(random_samples[:15], output_ids, prompt_length)
    p20, original,_ ,_ = tp_debug_log(random_samples[:20], output_ids, prompt_length)
    dfs.append({
        'name': file_path,
        'method': 'Random',
        'p_1': p1,
        'p_3': p3,
        'p_5': p5,
        'p_7': p7,
        'p_10': p10,
        'p_15': p15,
        'p_20': p20
    })
    trackstar_influence = np.load(f'data/trackstar/gutenberg/testing/gradients/influence_list_{file_path[:-3]}.npy')
    print(tokenizer.decode(ckpt['output_ids'][0]))
    sorted_indices = np.argsort(trackstar_influence)[::-1]
    samples = []
    for i in range(20):
        samples.append({
            'input_ids': torch.tensor([training_data[int(sorted_indices[i])]]),
            'attention_mask': torch.tensor([1] * len(training_data[int(sorted_indices[i])])),
            'labels': torch.tensor([training_data[int(sorted_indices[i])]])
        })
    p_1, original, _, _ = tp_debug_log(samples[:1], output_ids, prompt_length=ckpt['prompt_length'])
    p_3, _, _, _ = tp_debug_log(samples[:3], output_ids, prompt_length=ckpt['prompt_length'])
    p_5, _, _, _ = tp_debug_log(samples[:5], output_ids, prompt_length=ckpt['prompt_length'])
    p_7, _, _, _ = tp_debug_log(samples[:7], output_ids, prompt_length=ckpt['prompt_length'])
    p_10, _, _, _ = tp_debug_log(samples[:10], output_ids, prompt_length=ckpt['prompt_length'])
    p_15, _, _, _ = tp_debug_log(samples[:15], output_ids, prompt_length=ckpt['prompt_length'])
    p_20, _, _, _ = tp_debug_log(samples[:20], output_ids, prompt_length=ckpt['prompt_length'])
    dfs.append({
        'name': file_path,
        'method': 'TrackStar',
        'p_1': p_1,
        'p_3': p_3,
        'p_5': p_5,
        'p_7': p_7,
        'p_10': p_10,
        'p_15': p_15,
        'p_20': p_20,
        
    })
    output_ids = ckpt['output_ids']
    prompt_length = ckpt['prompt_length']
    
    query = tokenizer.decode(output_ids[0, prompt_length:], skip_special_tokens=True)
    print(query)
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    top_n = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:20]
    samples_bm25 = []
    for rank, (idx, score) in enumerate(top_n, 1):
        print(f"\nTop {rank} (index: {idx}, score: {score:.2f}):\n{corpus[idx][:300]}")
        print(tokenizer.decode(training_data[idx], skip_special_tokens=True))
        sample_bm25 = {
            'input_ids': torch.tensor(training_data[idx]),
            'attention_mask': torch.ones_like(torch.tensor(training_data[idx])),
            'labels': torch.tensor(training_data[idx])
        }
        samples_bm25.append(sample_bm25)

    p_1, original, _, _ = tp_debug_log(samples_bm25[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3, original, _, _ = tp_debug_log(samples_bm25[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5, original, _, _ = tp_debug_log(samples_bm25[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7, original, _, _ = tp_debug_log(samples_bm25[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10, original, _, _ = tp_debug_log(samples_bm25[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15, original, _, _ = tp_debug_log(samples_bm25[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20, original, _, _ = tp_debug_log(samples_bm25[:20],output_ids=output_ids, prompt_length=prompt_length)

    dfs.append({
        'name': file_path,
        'method': 'BM25',
        '1': p_1,
        '3': p_3,
        '5': p_5,
        '7': p_7,
        '10': p_10,
        '15': p_15,
        '20': p_20
    })
    
    total_df = pd.DataFrame(dfs)
    os.makedirs('data/results', exist_ok=True)
    total_df.to_csv('data/results/gutenberg_tailpatch_results.csv', index=False)
    

total_df = pd.DataFrame(dfs)
total_df.to_csv('data/results/gutenberg_tailpatch_results.csv', index=False)
total_df

In [ ]:
import pandas as pd
dfs = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for k, file_path in enumerate(ckpt_files):
    ckpt = torch.load(os.path.join(ckpt_dir, file_path), map_location=device)
    output_ids = ckpt['output_ids']
    prompt_length = ckpt['prompt_length']
    losses_ft = np.load(os.path.join(finetuned_dir, finetuned_losses[k]))
    losses_unl = np.load(os.path.join(unlearned_dir, unlearned_losses[k]))
    losses_diff = np.abs(losses_ft - losses_unl)
    sorted_indices = np.argsort(losses_diff)[::-1]
    print(tokenizer.decode(ckpt['output_ids'][0]))
    samples = []
    for i in range(20):
        samples.append({
            'input_ids': torch.tensor([training_data[int(sorted_indices[i])]]),
            'attention_mask': torch.tensor([1] * len(training_data[int(sorted_indices[i])])),
            'labels': torch.tensor([training_data[int(sorted_indices[i])]])
        })
    p_1, original, _, _ = tp_debug_log(samples[:1], output_ids, prompt_length=ckpt['prompt_length'])
    p_3, _, _, _ = tp_debug_log(samples[:3], output_ids, prompt_length=ckpt['prompt_length'])
    p_5, _, _, _ = tp_debug_log(samples[:5], output_ids, prompt_length=ckpt['prompt_length'])
    p_7, _, _, _ = tp_debug_log(samples[:7], output_ids, prompt_length=ckpt['prompt_length'])
    p_10, _, _, _ = tp_debug_log(samples[:10], output_ids, prompt_length=ckpt['prompt_length'])
    p_15, _, _, _ = tp_debug_log(samples[:15], output_ids, prompt_length=ckpt['prompt_length'])
    p_20, _, _, _ = tp_debug_log(samples[:20], output_ids, prompt_length=ckpt['prompt_length'])
    dfs.append({
        'name': file_path,
        'method': 'FT & UN',
        'p_1': p_1,
        'p_3': p_3,
        'p_5': p_5,
        'p_7': p_7,
        'p_10': p_10,
        'p_15': p_15,
        'p_20': p_20,
        
    })
    dfs.append({
        'name': file_path,
        'method': 'Original',
        'p_1': original,
        'p_3': original,
        'p_5': original,
        'p_7': original,
        'p_10': original,
        'p_15': original,
        'p_20': original,
    })
    random_samples_ = np.random.choice(len(training_data), size=20)
    random_samples = []
    for i in range(20):
        random_samples.append({
            'input_ids': torch.tensor(training_data[random_samples_[i]]),
            'attention_mask': torch.ones_like(torch.tensor(training_data[random_samples_[i]])),
            'labels': torch.tensor(training_data[random_samples_[i]])
        })
    p1, original,_ ,_ = tp_debug_log(random_samples[:1], output_ids, prompt_length)
    p3, original,_ ,_ = tp_debug_log(random_samples[:3], output_ids, prompt_length)
    p5, original,_ ,_ = tp_debug_log(random_samples[:5], output_ids, prompt_length)
    p7, original,_ ,_ = tp_debug_log(random_samples[:7], output_ids, prompt_length)
    p10, original,_ ,_ = tp_debug_log(random_samples[:10], output_ids, prompt_length)
    p15, original,_ ,_ = tp_debug_log(random_samples[:15], output_ids, prompt_length)
    p20, original,_ ,_ = tp_debug_log(random_samples[:20], output_ids, prompt_length)
    dfs.append({
        'name': file_path,
        'method': 'Random',
        'p_1': p1,
        'p_3': p3,
        'p_5': p5,
        'p_7': p7,
        'p_10': p10,
        'p_15': p15,
        'p_20': p20
    })
    trackstar_influence = np.load(f'data/training_data/trackstar/gutenberg/testing/gradients/influence_list_{file_path[:-3]}.npy')
    print(tokenizer.decode(ckpt['output_ids'][0]))
    sorted_indices = np.argsort(trackstar_influence)[::-1]
    samples = []
    for i in range(20):
        samples.append({
            'input_ids': torch.tensor([training_data[int(sorted_indices[i])]]),
            'attention_mask': torch.tensor([1] * len(training_data[int(sorted_indices[i])])),
            'labels': torch.tensor([training_data[int(sorted_indices[i])]])
        })
    p_1, original, _, _ = tp_debug_log(samples[:1], output_ids, prompt_length=ckpt['prompt_length'])
    p_3, _, _, _ = tp_debug_log(samples[:3], output_ids, prompt_length=ckpt['prompt_length'])
    p_5, _, _, _ = tp_debug_log(samples[:5], output_ids, prompt_length=ckpt['prompt_length'])
    p_7, _, _, _ = tp_debug_log(samples[:7], output_ids, prompt_length=ckpt['prompt_length'])
    p_10, _, _, _ = tp_debug_log(samples[:10], output_ids, prompt_length=ckpt['prompt_length'])
    p_15, _, _, _ = tp_debug_log(samples[:15], output_ids, prompt_length=ckpt['prompt_length'])
    p_20, _, _, _ = tp_debug_log(samples[:20], output_ids, prompt_length=ckpt['prompt_length'])
    dfs.append({
        'name': file_path,
        'method': 'TrackStar',
        'p_1': p_1,
        'p_3': p_3,
        'p_5': p_5,
        'p_7': p_7,
        'p_10': p_10,
        'p_15': p_15,
        'p_20': p_20,
        
    })
    output_ids = ckpt['output_ids']
    prompt_length = ckpt['prompt_length']
    
    query = tokenizer.decode(output_ids[0, prompt_length:], skip_special_tokens=True)
    print(query)
    tokenized_query = query.lower().split()
    scores = bm25.get_scores(tokenized_query)
    top_n = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:20]
    samples_bm25 = []
    for rank, (idx, score) in enumerate(top_n, 1):
        print(f"\nTop {rank} (index: {idx}, score: {score:.2f}):\n{corpus[idx][:300]}")
        print(tokenizer.decode(training_data[idx], skip_special_tokens=True))
        sample_bm25 = {
            'input_ids': torch.tensor(training_data[idx]),
            'attention_mask': torch.ones_like(torch.tensor(training_data[idx])),
            'labels': torch.tensor(training_data[idx])
        }
        samples_bm25.append(sample_bm25)

    p_1, original, _, _ = tp_debug_log(samples_bm25[:1],output_ids=output_ids, prompt_length=prompt_length)
    p_3, original, _, _ = tp_debug_log(samples_bm25[:3],output_ids=output_ids, prompt_length=prompt_length)
    p_5, original, _, _ = tp_debug_log(samples_bm25[:5],output_ids=output_ids, prompt_length=prompt_length)
    p_7, original, _, _ = tp_debug_log(samples_bm25[:7],output_ids=output_ids, prompt_length=prompt_length)
    p_10, original, _, _ = tp_debug_log(samples_bm25[:10],output_ids=output_ids, prompt_length=prompt_length)
    p_15, original, _, _ = tp_debug_log(samples_bm25[:15],output_ids=output_ids, prompt_length=prompt_length)
    p_20, original, _, _ = tp_debug_log(samples_bm25[:20],output_ids=output_ids, prompt_length=prompt_length)

    dfs.append({
        'name': file_path,
        'method': 'BM25',
        '1': p_1,
        '3': p_3,
        '5': p_5,
        '7': p_7,
        '10': p_10,
        '15': p_15,
        '20': p_20
    })
    
    total_df = pd.DataFrame(dfs)
    os.makedirs('data/results', exist_ok=True)
    total_df.to_csv('data/results/gutenberg_tailpatch_results.csv', index=False)
    

total_df = pd.DataFrame(dfs)
total_df.to_csv('data/results/gutenberg_tailpatch_results.csv', index=False)
total_df

In [ ]:
df = pd.read_csv('../data/results/gutenberg_tailpatch_results_all.csv')
df_exp = df.copy()
df_exp[df_exp.select_dtypes(include=[np.number]).columns] = \
    df_exp.select_dtypes(include=[np.number]).applymap(lambda x: np.exp(x-1))

df = df_exp

In [ ]:
import pandas as pd
import numpy as np


cols_to_check = ['1', '3', '5', '7', '10', '15', '20']
df.columns = df.columns.astype(str)

results = []

for namegrouped, group in df.groupby('name'):
    original_row = group[group['method'].str.lower() == 'original']
    if original_row.empty:
        print(namegrouped)
        continue
    original_values = original_row[cols_to_check].iloc[0]
    
    
    for _, row in group.iterrows():
        if row['method'].lower() == 'original':
            continue
        rel_increase = np.abs((row[cols_to_check].astype(float) - original_values.astype(float)) /np.abs(original_values.astype(float))) * 100
        result_row = {
            'name': namegrouped,
            'method': row['method']
        }
        for col in cols_to_check:
            result_row[f'rel_increase_{col}'] = rel_increase[col]
        results.append(result_row)

# Final result
rel_df = pd.DataFrame(results)

rel_df.loc[rel_df['name']=='William Shakespeare_2.pt']

In [ ]:
avg_df = (
    rel_df.groupby('method')
    .agg({
        'rel_increase_1': ['mean', 'std'],
        'rel_increase_3': ['mean', 'std'],
        'rel_increase_5': ['mean', 'std'],
        'rel_increase_7': ['mean', 'std'],
        'rel_increase_10': ['mean', 'std'],
        'rel_increase_15': ['mean', 'std'],
        'rel_increase_20': ['mean', 'std'],
    })
)
avg_df = avg_df.round(2)
avg_df



In [ ]:
from scipy.stats import ttest_ind

columns = ['rel_increase_1', 'rel_increase_3', 'rel_increase_5', 
           'rel_increase_7', 'rel_increase_10', 'rel_increase_15', 'rel_increase_20']

for col in columns:
    ft_values = rel_df[rel_df['method'] == 'FT & UN'][col]
    trackstar_values = rel_df[rel_df['method'] == 'TrackStar'][col]
    
    t_stat, p_val = ttest_ind(ft_values, trackstar_values, equal_var=False)
    print(f"{col}: t = {t_stat:.3f}, p = {p_val:.4f}")


In [ ]:
import pandas as pd
df_gecko = pd.read_csv('../data/results/results_aggregated_gecko_gutenberg.csv')
df_gecko

In [ ]:
import pandas as pd
import numpy as np


cols_to_check = ['1', '3', '5', '7', '10', '15', '20']
df = df_gecko.copy()
df.columns = df.columns.astype(str)

results = []




for namegrouped, group in df.groupby('name'):
    original_row = group[group['method'].str.lower() == 'original']
    if original_row.empty:
        print(namegrouped)
        continue
    original_values = original_row[cols_to_check].iloc[0]
    
    
    for _, row in group.iterrows():
        if row['method'].lower() == 'original':
            continue
        rel_increase = np.abs((row[cols_to_check].astype(float) - original_values.astype(float)) /np.abs(original_values.astype(float))) * 100
        result_row = {
            'name': namegrouped,
            'method': row['method']
        }
        for col in cols_to_check:
            result_row[f'rel_increase_{col}'] = rel_increase[col]
        results.append(result_row)

# Final result
rel_df = pd.DataFrame(results)


In [ ]:
avg_df_gecko = (
    rel_df.groupby('method')
    .agg({
        'rel_increase_1': ['mean', 'std'],
        'rel_increase_3': ['mean', 'std'],
        'rel_increase_5': ['mean', 'std'],
        'rel_increase_7': ['mean', 'std'],
        'rel_increase_10': ['mean', 'std'],
        'rel_increase_15': ['mean', 'std'],
        'rel_increase_20': ['mean', 'std'],
    })
)
avg_df_gecko = avg_df_gecko.round(2)
avg_df_gecko



In [ ]:
combined_df = pd.concat([avg_df_gecko, avg_df])
combined_df

In [ ]:
import pandas as pd
df_cleaned = combined_df.loc[:, combined_df.columns.get_level_values(1) == 'mean']
df_cleaned.columns = [col[0].split('_')[-1] for col in df_cleaned.columns]

df_cleaned.columns.name = None
df_cleaned.index.name = None
df_cleaned = df_cleaned.sort_values(by=['1'], ascending=True)
df_cleaned.rename(index={'FT/UN': 'FT & UN'}, inplace=True)
#df_cleaned.to_csv('avg_df_fisher_raw_.csv', index=True)
df_cleaned


In [ ]:
df_cleaned.to_csv('../data/results/avg_df_gutenberg_gecko.csv', index=True)

In [ ]:
import pandas as pd 
import wandb
api = wandb.Api()

runs = api.runs("Retraining Gutenberg FTUN")

summary_list, config_list, name_list = [], [], []

history_df = {}
authors = ["Arthur Christopher Benson_2", "Carolyn Wells_2",
             "Jane Austen_2", "Edward Payson Roe_2" ]
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})  
    name_list.append(run.name)
    
    config = run.config
    author = config.get("author", "N/A")
    if author not in authors:
        continue
    num_samples = config.get("num_samples", "N/A")
    mode = config.get("mode", "N/A")
    if len(run.history()) > 20:
        print(run.name)
        print(f"Run: {run.name}")
        print(f"Author: {author}, Num Samples: {num_samples}, Mode: {mode}")
        history_df[(author, num_samples, mode)] = run.history()


In [ ]:
losses_for_bm25_50 = []
losses_for_bm25_100 = []
losses_for_bm25_20 = []
losses_for_attributed_50 = []
losses_for_attributed_100 = []
losses_for_attributed_20 = []
losses_for_trackstar_20 = []
losses_for_trackstar_50 = []
losses_for_trackstar_100 = []
losses_for_gecko_20 = []
losses_for_gecko_50 = []
losses_for_gecko_100 = []
for run in history_df.keys():
    if run[2] == 'bm25':
        masked_loss = history_df[run]
        last_loss = masked_loss["Masked Loss"].dropna().iloc[-1]
        if run[1] == 50:
            losses_for_bm25_50.append(last_loss)
        elif run[1] == 100:
            losses_for_bm25_100.append(last_loss)
        elif run[1] == 20:
            losses_for_bm25_20.append(last_loss)
    if run[2] == 'attributed':
        masked_loss = history_df[run]
        last_loss = masked_loss["Masked Loss"].dropna().iloc[-1]
        if run[1] == 50:
            losses_for_attributed_50.append(last_loss)
        elif run[1] == 100:
            losses_for_attributed_100.append(last_loss)
        elif run[1] == 20:
            losses_for_attributed_20.append(last_loss)
    if run[2] == 'trackstar':
        masked_loss = history_df[run]
        last_loss = masked_loss["Masked Loss"].dropna().iloc[-1]
        if run[1] == 50:
            losses_for_trackstar_50.append(last_loss)
        elif run[1] == 100:
            losses_for_trackstar_100.append(last_loss)
        elif run[1] == 20:
            losses_for_trackstar_20.append(last_loss)

    if run[2] == 'gecko':
        masked_loss = history_df[run]
        last_loss = masked_loss["Masked Loss"].dropna().iloc[-1]
        if run[1] == 50:
            losses_for_gecko_50.append(last_loss)
        elif run[1] == 100:
            losses_for_gecko_100.append(last_loss)
        elif run[1] == 20:
            losses_for_gecko_20.append(last_loss)

print(len(losses_for_bm25_50))
print(len(losses_for_bm25_100))
print(len(losses_for_bm25_20))
print(len(losses_for_gecko_50))
print(losses_for_bm25_50)
print(losses_for_bm25_100)
print(losses_for_bm25_20)
print(losses_for_trackstar_20)
print(losses_for_trackstar_50)
print(losses_for_trackstar_100)


In [ ]:
import numpy as np
avgs_bm25 = [np.mean(np.array(losses_for_bm25_20)), np.mean(np.array(losses_for_bm25_50)), np.mean(np.array(losses_for_bm25_100))]
avgs_attributed = [np.mean(np.array(losses_for_attributed_20)), np.mean(np.array(losses_for_attributed_50)), np.mean(np.array(losses_for_attributed_100))]
avgs_trackstar = [np.mean(np.array(losses_for_trackstar_20)), np.mean(np.array(losses_for_trackstar_50)), np.mean(np.array(losses_for_trackstar_100))]
avgs_gecko = [np.mean(np.array(losses_for_gecko_20)), np.mean(np.array(losses_for_gecko_50)), np.mean(np.array(losses_for_gecko_100))]
print(avgs_bm25)
print(avgs_attributed)
print(avgs_trackstar)
print(avgs_gecko)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Averages
avgs_bm25 = [np.mean(np.array(losses_for_bm25_20)), np.mean(np.array(losses_for_bm25_50)), np.mean(np.array(losses_for_bm25_100))]
avgs_attributed = [np.mean(np.array(losses_for_attributed_20)), np.mean(np.array(losses_for_attributed_50)), np.mean(np.array(losses_for_attributed_100))]
avgs_trackstar = [np.mean(np.array(losses_for_trackstar_20)), np.mean(np.array(losses_for_trackstar_50)), np.mean(np.array(losses_for_trackstar_100))]

x_vals = [20, 50, 100]

In [ ]:

original_losses = []
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('../out/gpt2-scratch-mixed')
model.eval()
for author in authors:
    ckpt = torch.load(f'samples/{author}.pt', map_location='cpu')
    sentence = ckpt['prompt'] + ckpt['sentence']
    output_ids = tokenizer.encode(sentence, return_tensors='pt')
    prompt_length = len(tokenizer.encode(ckpt['prompt']))
    print(tokenizer.decode(output_ids[0], skip_special_tokens=True))
    labels = output_ids.clone()
    labels[..., :prompt_length] = -100
    attention_mask = torch.ones_like(output_ids)
    outputs = model(output_ids, labels=labels, attention_mask=attention_mask)
    loss = outputs.loss
    original_losses.append(loss.item())
    print(loss.item())
print(original_losses)

import matplotlib.pyplot as plt
plt.figure(figsize=(10, 5))
plt.plot(original_losses)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Averages
avgs_bm25 = [np.mean(original_losses)] + [
    np.mean(np.array(losses_for_bm25_20)),
    np.mean(np.array(losses_for_bm25_50)),
    np.mean(np.array(losses_for_bm25_100)),
]
avgs_attributed = [np.mean(original_losses)] + [
    np.mean(np.array(losses_for_attributed_20)),
    np.mean(np.array(losses_for_attributed_50)),
    np.mean(np.array(losses_for_attributed_100)),
]
avgs_trackstar = [np.mean(original_losses)] + [
    np.mean(np.array(losses_for_trackstar_20)),
    np.mean(np.array(losses_for_trackstar_50)),
    np.mean(np.array(losses_for_trackstar_100)),
]
avgs_gecko = [np.mean(original_losses)] + [
    np.mean(np.array(losses_for_gecko_20)),
    np.mean(np.array(losses_for_gecko_50)),
    np.mean(np.array(losses_for_gecko_100)),
]
# X-axis values (without initial dummy 0)
x_vals = [0,20, 50, 100]

# Plot
plt.figure(figsize=(13, 7))  # was (8, 5) before — increase height

plt.plot(x_vals, avgs_attributed, marker='o', label='Attributed')
plt.plot(x_vals, avgs_bm25, marker='s', label='BM25')
plt.plot(x_vals, avgs_trackstar, marker='^', label='Trackstar')
plt.plot(x_vals, avgs_gecko, marker='^', label='Gecko')
plt.xlim(0.5, 120)
plt.ylim(0, 6)
plt.xticks([0, 20, 50, 100])
plt.xlabel("Top-k Removed")
plt.ylabel("Average Loss after Retraining")
plt.title("Average Loss vs Top-k Removed")
plt.legend()

plt.tight_layout()
plt.savefig('retraining_figure.pdf')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# Style settings
mpl.rcParams.update({
    "font.size": 9,
    "font.family": "times new roman",
    "axes.labelsize": 9,
    "axes.titlesize": 10,
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.dpi": 300
})

# Plot dimensions
fig_width = 3.25  # inches for one column
fig_height = fig_width * 0.8
plt.figure(figsize=(fig_width, fig_height))

# Data
x_vals = [0, 20, 50, 100]
plt.plot(x_vals, avgs_gecko, marker='^', label='GECKO')
plt.plot(x_vals, avgs_bm25, marker='s', label='BM25')
plt.plot(x_vals, avgs_attributed, marker='o', label='DABGO')
plt.plot(x_vals, avgs_trackstar, marker='^', label='TRACKSTAR')


plt.xlabel("Top-k Removed")
plt.ylabel("Loss After Retraining")
#plt.title("Loss After Retraining Without Top-k", pad=10)


# Legend inside plot, top-centered
plt.legend(
    loc='lower center',
    bbox_to_anchor=(0.5, 1),
    ncol=3,
    frameon=True,
    fancybox=True,
    framealpha=0.9,
    edgecolor='black',
    borderpad=0.6
)

# Axis settings
plt.xlim(0, 110)
plt.ylim(0, 6)
plt.xticks([0, 20, 50, 100])
plt.yticks(np.arange(0, 6,1))  # or even step=0.25 if clarity improves

# Layout and save
plt.tight_layout(pad=0.5)
plt.savefig("retrained_losses.pdf", format="pdf", bbox_inches="tight")
plt.show()


# Interpretability

In [ ]:
import torch
import torch.nn.functional as F

def get_token_losses(model, input_ids, device, pad_token_id):
    """
    Compute per-token losses (negative log-likelihoods) for a single input sequence.
    Returns a 1D tensor of length (seq_len - 1), excluding the first token.
    """
    model.to(device)
    model.eval()
    input_ids = input_ids.to(device)  # shape: (1, seq_len)

    attention_mask = (input_ids != pad_token_id).long().to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # shape: (1, seq_len, vocab_size)
        # Shift for next-token prediction
        shift_logits = logits[:, :-1, :]
        shift_labels = input_ids[:, 1:]
        log_probs = F.log_softmax(shift_logits, dim=-1)
        per_token_log_probs = log_probs.gather(dim=-1, index=shift_labels.unsqueeze(-1)).squeeze(-1)
        per_token_nll = -per_token_log_probs  # shape: (1, seq_len - 1)
    return per_token_nll.squeeze(0)  # shape: (seq_len - 1,)


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

model_ft = GPT2LMHeadModel.from_pretrained('out/wiki_model')
ckpt_ft = torch.load('out/wiki_models_finetuned/fisher_regularized_models/ww2_finetuned_fisher.pt')
ckpt_unl = torch.load('out/wiki_models_unlearned/fisher_regularized_models/ww2_unlearned_fisher.pt')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token_id = tokenizer.eos_token_id
model_ft.load_state_dict(ckpt_ft['model'])
model_ft.to('cuda')
model_ft.eval()
model_unl = GPT2LMHeadModel.from_pretrained('out/wiki_model')
model_unl.load_state_dict(ckpt_unl['model'])
model_unl.to('cuda')
model_unl.eval()
print('Model loaded')


In [ ]:
sample_text = """
Japan's attack on Pearl Harbor took place on December 7, 1941. The U.S. military suffered 18 ships damaged or sunk, and 2,400 people were killed. Its most significant consequence was the entrance of the United States into World War II. The US had previously been neutral but subsequently entered the Pacific War, the Battle of the Atlantic and the European theatre of war. Following the attack, the US interned 120,000 Japanese Americans, 11,000 German Americans, and 3,000 Italian Americans.
"""
sample_ids = tokenizer.encode(sample_text, return_tensors='pt')
prompt_length = ckpt_ft['prompt_length']
losses_ft = get_token_losses(model_ft, sample_ids, 'cuda', tokenizer.pad_token_id)
losses_unl = get_token_losses(model_unl, sample_ids, 'cuda', tokenizer.pad_token_id)


In [ ]:
import torch
import numpy as np
from transformers import GPT2Tokenizer

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token_id = tokenizer.eos_token_id

# 30 highlight macros from strong to light blue
hl_macros = [
    f'\\hl{chr(c)}' for c in range(ord('A'), ord('Z') + 1)
] + ['\\hlAA', '\\hlAB', '\\hlAC', '\\hlAD']  # Total = 30

# Compute per-token loss difference
token_diff = np.abs(losses_ft.cpu().numpy() - losses_unl.cpu().numpy())
input_ids = sample_ids.squeeze(0)
# Get top 30 most changed tokens
top_token_indices = token_diff.argsort()[::-1][:25]

# Create mapping from token index (+1 offset) to LaTeX macro
highlight_rank = {i + 1: hl_macros[rank] for rank, i in enumerate(top_token_indices)}

# Highlighted token sequence
highlighted_tokens = []
for i, tid in enumerate(input_ids):
    token_str = tokenizer.decode([tid]).replace("{", "\\{").replace("}", "\\}")
    if tid == tokenizer.pad_token_id:
        continue
    if i in highlight_rank:
        macro = highlight_rank[i]
        highlighted_tokens.append(f"{macro}{{{token_str}}}")
    else:
        highlighted_tokens.append(token_str)

# Output LaTeX-friendly string
highlighted_sentence = "".join(highlighted_tokens)

print(highlighted_sentence)
